In [1]:
import pandas as pd
import re
from DataCollector import DataCollector
DATA_DIR = "./src/data"

In [2]:
collector = DataCollector(DATA_DIR)

In [3]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, XSD

In [4]:
# Criar o modelo (grafo RDF)
model = Graph()

# Definir namespaces 
base = Namespace("http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#")
xsd  = Namespace("http://www.w3.org/2001/XMLSchema#")
foaf = Namespace("http://xmlns.com/foaf/0.1/")
rdfs = Namespace("http://www.w3.org/2000/01/rdf-schema#")
rdf  = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
dbo  = Namespace("http://dbpedia.org/ontology/")
dbp  = Namespace("http://dbpedia.org/property/")
dbr  = Namespace("http://dbpedia.org/resource/")

# Vincular os namespaces ao grafo 
model.bind("", base)
model.bind("xsd", xsd)
model.bind("foaf", foaf)
model.bind("rdf", rdf)
model.bind("rdfs", rdfs)
model.bind("dbo", dbo)
model.bind("dbp", dbp)
model.bind("dbr", dbr)

In [5]:
def add_countries_to_model():
    countries = collector.get_countries()

    for country_name in countries:
        country_name_formated = re.sub(r"[^a-zA-Z0-9]", "", country_name)
        country = URIRef(base + country_name_formated)
        model.add((country, RDF.type, dbo.Country))
        model.add((country, foaf.name, Literal(country_name)))

In [6]:
def add_styles_to_model():
    artist_styles = collector.get_artist_styles()
    work_styles = collector.get_work_styles()
    styles = set(artist_styles + work_styles)

    for style_name in styles:
        style_name_formated = "Style_"+re.sub(r"[^a-zA-Z0-9]", "", style_name)
        style = URIRef(base + style_name_formated)
        model.add((style, RDF.type, base.Style))
        model.add((style, foaf.name, Literal(style_name)))

In [7]:
def add_subjects_to_model():
    subjects = collector.get_subjects()

    for subject_name in subjects:
        subject_name_formated = 'Subject_'+re.sub(r"[^a-zA-Z0-9]", "", subject_name)
        subject = URIRef(base + subject_name_formated)
        model.add((subject, RDF.type, base.Subject))
        model.add((subject, foaf.name, Literal(subject_name)))

In [8]:
def add_nationalities_to_model():
    nationalities = collector.get_artist_nationalities()

    for nationality_name in nationalities:
        nationality_name_formated = re.sub(r"[^a-zA-Z0-9]", "", nationality_name)
        nationality = URIRef(base + nationality_name_formated)
        model.add((nationality, RDF.type, base.Nationality))
        model.add((nationality, foaf.name, Literal(nationality_name)))

In [9]:
def add_museums_to_model():
    data = collector.get_museums()
    tam = data.shape[0]
    
    for i in range (tam):
        museum_identifier = data.iloc[i]["identifier"]
        museum = URIRef(base + museum_identifier)
        model.add((museum, RDF.type, dbo.Museum))

        museum_name = str(data.iloc[i]["name"])
        model.add((museum, foaf.name, Literal(museum_name)))

        country_name = re.sub(r"[^a-zA-Z0-9]", "", str(data.iloc[i]["country"]))
        if country_name != "nan":
            model.add((museum, dbp.country, URIRef(base + country_name )))

        state = str(data.iloc[i]["state"])
        if state != "nan":
            model.add((museum, base.state, Literal(state)))
        
        city = str(data.iloc[i]["city"])
        if city != "nan":
            model.add((museum, base.city, Literal(city)))
        
        address = str(data.iloc[i]["address"])
        if address != "nan":
            model.add((museum, base.address, Literal(address)))
        
        postal = str(data.iloc[i]["postal"])
        if postal != "nan":
            model.add((museum, base.postal, Literal(postal)))
        
        phone = str(data.iloc[i]["phone"])
        if phone != "nan":
            model.add((museum, base.phone, Literal(phone)))
        
        url = str(data.iloc[i]["url"])
        if url != "nan":
            model.add((museum, foaf.homepage, Literal(url)))

In [10]:
def add_artists_to_model():
    data = collector.get_artists()
    tam = data.shape[0]
    
    for i in range (tam):
        artist_identifier = data.iloc[i]["identifier"]
        artist = URIRef(base + artist_identifier)
        model.add((artist, RDF.type, dbo.Artist))

        full_name = str(data.iloc[i]["full_name"])
        model.add((artist, foaf.name, Literal(full_name)))

        first_name = str(data.iloc[i]["first_name"])
        if first_name != "nan":
            model.add((artist, foaf.firstName, Literal(first_name)))

        last_name = str(data.iloc[i]["last_name"])
        if last_name != "nan":
            model.add((artist, foaf.lastName, Literal(last_name)))
        
        birth = str(data.iloc[i]["birth"])
        if birth != "nan":
            model.add((artist, base.birth, Literal(birth)))

        death = str(data.iloc[i]["death"])
        if death != "nan":
            model.add((artist, base.death, Literal(death)))
        
        nationality = re.sub(r"[^a-zA-Z0-9]", "", str(data.iloc[i]["nationality"]))
        if nationality != "nan":
            model.add((artist, dbp.nationality, URIRef(base + nationality )))
        
        style = re.sub(r"[^a-zA-Z0-9]", "", str(data.iloc[i]["style"]))
        if style != "nan":
            model.add((artist, dbo.movement, URIRef(base + "Style_" + style )))

In [11]:
def add_works_to_model():
    data = collector.get_works()
    tam = data.shape[0]

    for i in range (tam):
        work_identifier = data.iloc[i]["identifier"]
        work = URIRef(base + work_identifier)
        model.add((work, RDF.type, dbo.Painting))

        name = str(data.iloc[i]["name"])
        model.add((work, foaf.name, Literal(name)))

        artist = collector.get_artist_by_id(data.iloc[i]["artist_id"])
        if artist is not None:
            model.add((work, dbp.artist, URIRef(base + artist )))
        
        style = re.sub(r"[^a-zA-Z0-9]", "", str(data.iloc[i]["style"]))
        if style != "nan":
            model.add((work, dbp.style, URIRef(base + "Style_" + style )))
        
        subject = collector.get_subject_by_id(data.iloc[i]["work_id"])
        if subject is not None:
            model.add((work, dbp.subject, URIRef(base + subject )))

        museum_id = data.iloc[i]["museum_id"]
        if museum_id != "nan":
            work_museum = collector.get_museum_by_id(museum_id)
            if work_museum is not None:
                model.add((work, dbp.museum, URIRef(base + work_museum)))
        

In [12]:
add_nationalities_to_model()
add_subjects_to_model()
add_countries_to_model()
add_styles_to_model()
add_works_to_model()
add_museums_to_model()
add_artists_to_model()

In [13]:
# Serializar o grafo para um arquivo ou string
output_file = "output.ttl"  
with open(output_file, "w", encoding="utf-8") as fout:
    fout.write(model.serialize(format="turtle"))

In [14]:
from owlrl import DeductiveClosure, RDFS_Semantics

In [15]:
schema_file = "./paintingsontology.ttl"
data_file = "./output.ttl"

schema = Graph()
schema.parse(schema_file)  # Ler o esquema

data = Graph()
data.parse(data_file, format="turtle")  # Ler os dados 

inferred_model = Graph()
inferred_model += schema
inferred_model += data

DeductiveClosure(RDFS_Semantics).expand(inferred_model)  # Inferência RDFS

print("Modelo inferido:")
output_file2 = "output2.ttl"  
with open(output_file2, "w", encoding='utf-8') as fout:
    fout.write(inferred_model.serialize(format="turtle"))

Modelo inferido:


In [16]:
museums = collector.get_museums()


In [17]:
museums_name = museums["name"].tolist()

In [18]:
from geopy.geocoders import Nominatim
import random

def obter_lat_lon_nominatim(endereco):
    geolocator = Nominatim(user_agent="sua_aplicacao")
    localizacao = geolocator.geocode(endereco)
    if localizacao:
        return localizacao.latitude, localizacao.longitude
    else:
        print("Endereço não encontrado")
        return None, None

# Exemplo de uso
elemento_aleatorio = random.choice(museums_name)
endereco = random.choice(museums_name)
latitude, longitude = obter_lat_lon_nominatim(endereco)
print(f"Latitude: {latitude}, Longitude: {longitude}")

import folium

# create tuples representing our location
location = float(latitude), float(longitude)

# center the map 
center = (latitude, longitude)

# create a Folium map centred at the above location
m = folium.Map(location=center, zoom_start=50, width=800, height=400)

# add markers at the locations
folium.Marker(location, popup="The postcode brought me here").add_to(m)

# refer to the map to display it in Jupyter/Colab notebooks
m

Latitude: 51.48077585, Longitude: -0.005122450776572108


In [19]:
# Pesquisando dados de artistas por inicio do nome
searchQuery = 'Ad'.lower()
qres = inferred_model.query(
    """
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?artistName ?birth ?death ?style ?nationality
        WHERE {
            ?artist
                rdf:type dbo:Artist ;
                foaf:name ?artistName ;
            .
            OPTIONAL {?artist :birth ?birth .}
            OPTIONAL {?artist :death ?death .}
            OPTIONAL {?artist dbo:movement/foaf:name ?style .}
            OPTIONAL {?artist dbp:nationality/foaf:name ?nationality .}
            FILTER (STRSTARTS(LCASE(STR(?artistName)), ?searchQuery))
        }
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
        'dbo': dbo,
        'dbp': dbp,
    },
    initBindings={'searchQuery': Literal(searchQuery)},
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

Adolph Ulrich Wertmuller | 1751 | 1811 | Portraitist | Swiss
Adèle Romany | 1769 | 1846 | Portraitist | French
Adélaïde Labille-Guiard | 1749 | 1803 | Neoclassical | French
Adriaen Thomasz. Key | 1544 | 1589 | Renaissance | Dutch
Adriaan De Lelie | 1755 | 1820 | Portraitist | Dutch


In [20]:
# Pesquisando dados de museus por inicio do nome
searchQuery = 'Mus'.lower()
qres = inferred_model.query(
    """
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?museumName ?country ?state ?city ?address ?postal ?phone ?url
        WHERE {
            ?museum
                rdf:type dbo:Museum ;
                foaf:name ?museumName ;
            .
            OPTIONAL {?museum dbp:country/foaf:name ?country .}
            OPTIONAL {?museum :state ?state .}
            OPTIONAL {?museum :city ?city .}
            OPTIONAL {?museum :address ?address .}
            OPTIONAL {?museum :postal ?postal .}
            OPTIONAL {?museum :phone ?phone .}
            OPTIONAL {?museum foaf:homepage ?url .}
            FILTER (STRSTARTS(LCASE(STR(?museumName)), ?searchQuery))
        }
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
        'dbo': dbo,
        'dbp': dbp,
    },
    initBindings={'searchQuery': Literal(searchQuery)},
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

Museum Folkwang | Germany | Essen | 45128 | Museumsplatz 1 | NULL | 49 201 8845000 | https://www.museum-folkwang.de/en
Musée des Beaux-Arts Quimper | France | Quimper | 29000 | 40 Pl. Saint-Corentin | NULL | 33 2 98 95 45 20 | https://www.mbaq.fr/en/home-3.html
Museum of Fine Arts | Switzerland | NULL | Bern | Hodlerstrasse 8 | 3011 | +41 31 328 09 44 | https://www.kunstmuseumbern.ch/en/
Musée du Louvre | France | Paris | 75001 | Rue de Rivoli | NULL | 33 1 40 20 50 50 | https://www.louvre.fr/en
Musée Marmottan Monet | France | NULL | Paris | 2 Rue Louis Boilly | 75016 | +33 1 44 96 50 33 | https://www.marmottan.fr/en/
Musée d'Orsay | France | NULL | Paris | 1 Rue de la Légion d'Honneur | 75007 | +33 (0)1 40 49 48 14 | https://www.musee-orsay.fr/en
Museo del Prado | Spain | NULL | Madrid | C. de Ruiz de Alarcón, 23 | 28014 | +34 913 30 28 00 | https://www.museodelprado.es/en
Museum of Fine Arts, Houston | USA | TX | Houston | 1001 Bissonnet St | 77005 | +1 713 639-7300 | https://www.mf

In [21]:
# Pesquisando dados de obras por inicio do nome
searchQuery = 'The Grand'.lower()
qres = inferred_model.query(
    """
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?workName ?artist ?style ?subject ?location
        WHERE {
            ?work
                rdf:type dbo:Painting ;
                foaf:name ?workName ;
            .
            OPTIONAL {?work dbp:artist/foaf:name ?artist .}
            OPTIONAL {?work dbp:style/foaf:name ?style .}
            OPTIONAL {?work dbp:subject/foaf:name ?subject .}
            OPTIONAL {?work dbp:museum/foaf:name ?location .}
            FILTER (STRSTARTS(LCASE(STR(?workName)), ?searchQuery))
        }
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
        'dbo': dbo,
        'dbp': dbp,
    },
    initBindings={'searchQuery': Literal(searchQuery)},
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

The Grand Canal in Venice | Albert Marquet | Post-Impressionism | Rivers/Lakes | NULL
The Grand Canal Above the Rialto | Francesco Guardi | Rococo | Architectures | The Metropolitan Museum of Art
The Grand Canal | Claude Monet | Impressionism | Rivers/Lakes | Museum of Fine Arts Boston
The Grand Bath at Bursa | Jean-Léon Gérôme | Orientalism | Nude | NULL
The Grand Canal in Venice from Palazzo Flangini to Campo San Marcuola | Canaletto | Rococo | Architectures | The J. Paul Getty Museum
The Grand Canal, Venice | Joseph M. W. Turner | Neo-Classicism | Rivers/Lakes | The Metropolitan Museum of Art
The Grand Canyon of the Colorado | Thomas Moran | American Landscape | Landscape Art | NULL
The Grand White Eunuch | Jean-Léon Gérôme | Orientalism | NULL | NULL
The Grand Canal from Palazzo Flangini to Campo San Marcuola | Canaletto | Rococo | Architectures | NULL
The Grande Creuse at Pont de Vervy | Claude Monet | Impressionism | NULL | Philadelphia Museum of Art
The Grands Boulevards | Pierr

In [22]:
# Pesquisando dados de estilos por inicio do nome
searchQuery = 'America'.lower()
qres = inferred_model.query(
    """
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?styleName
        WHERE {
            ?style
                rdf:type :Style ;
                foaf:name ?styleName ;
            .
            FILTER (STRSTARTS(LCASE(STR(?styleName)), ?searchQuery))
        }
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
    },
    initBindings={'searchQuery': Literal(searchQuery)},
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

American West
American Art
America West
American Landscape


In [23]:
# Pesquisando dados de temas por inicio do nome
searchQuery = 'A'.lower()
qres = inferred_model.query(
    """
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?subjectName
        WHERE {
            ?subject
                rdf:type :Subject ;
                foaf:name ?subjectName ;
            .
            FILTER (STRSTARTS(LCASE(STR(?subjectName)), ?searchQuery))
        }
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
    },
    initBindings={'searchQuery': Literal(searchQuery)},
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

Abstract/Modern Art
Architectures
Autumn/Fall


In [24]:
# Listar 10 obras de um museu junto com o estilo daquela obra
museumURL = 'http://www.clevelandart.org/'
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        SELECT ?title ?style
        WHERE {{
            SERVICE <https://dbpedia.org/sparql> {{
                ?workDbo a dbo:Work ;
                         dbp:title ?title ;
                         dbo:museum/dbp:website <{museumURL}> .
            }}
            OPTIONAL {{
                ?workLocal
                    rdf:type dbo:Painting ;
                    foaf:name ?titleLocal ;
                    dbp:style/foaf:name ?style ;
                .
                FILTER (STR(?title) = STR(?titleLocal))
            }}
        }}
        LIMIT 10
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
        'dbr': dbr,
        'dbo': dbo,
        'dbp': dbp,
    }
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

Bottle, Glass, Fork | NULL
Stag at Sharkey's | NULL
Constellations | NULL
The Large Plane Trees | NULL
Cleveland Apollo | NULL
Fall of Simon Magus | NULL
The Crucifixion of Saint Andrew | Baroque
Un lièvre et un gigot de mouton | NULL
La Vie | NULL
Source | NULL


In [25]:
# Retornar os dados e a imagem de um artista pelo nome
artistName = 'Adélaïde Labille-Guiard'
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?birth ?death ?style ?nationality ?image
        WHERE {{
            OPTIONAL {{
                SERVICE <https://dbpedia.org/sparql> {{
                    ?artistDbo a dbo:Animal ;
                        rdfs:label "{artistName}"@en ;
                        dbo:thumbnail ?image .
                }}
            }}
            ?artist
                rdf:type dbo:Artist ;
                foaf:name "{artistName}" ;
            .
            OPTIONAL {{?artist :birth ?birth .}}
            OPTIONAL {{?artist :death ?death .}}
            OPTIONAL {{?artist dbo:movement/foaf:name ?style .}}
            OPTIONAL {{?artist dbp:nationality/foaf:name ?nationality .}}
        }}
        LIMIT 1
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
        'dbr': dbr,
        'dbo': dbo,
        'dbp': dbp,
        'rdfs': rdfs,
    },
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

1749 | 1803 | Neoclassical | French | http://commons.wikimedia.org/wiki/Special:FilePath/Adélaïde_Labille-Guiard_-_Self-Portrait_with_Two_Pupils_-_The_Metropolitan_Museum_of_Art.jpg?width=300


In [26]:
# Retornar os dados e as coordenadas de um museu pelo nome
museumName = 'National Maritime Museum'
qres = inferred_model.query(
    f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        
        SELECT ?country ?state ?city ?address ?postal ?phone ?url ?coords
        WHERE {{
            OPTIONAL {{
                SERVICE <https://dbpedia.org/sparql> {{
                    ?museumDbo a dbo:Building ;
                        rdfs:label "{museumName}"@en ;
                        georss:point ?coords .
                }}
            }}
            ?museum
                rdf:type dbo:Museum ;
                foaf:name "{museumName}" ;
            .
            OPTIONAL {{?museum dbp:country/foaf:name ?country .}}
            OPTIONAL {{?museum :state ?state .}}
            OPTIONAL {{?museum :city ?city .}}
            OPTIONAL {{?museum :address ?address .}}
            OPTIONAL {{?museum :postal ?postal .}}
            OPTIONAL {{?museum :phone ?phone .}}
            OPTIONAL {{?museum foaf:homepage ?url .}}
        }}
        LIMIT 1
    """,
    initNs={
        'rdf': rdf,
        'foaf': foaf,
        'dbr': dbr,
        'dbo': dbo,
        'dbp': dbp,
        'rdfs': rdfs,
        'georss': 'http://www.georss.org/georss/',
    },
)

for row in qres:
    print(' | '.join([e or 'NULL' for e in row]))

United Kingdom | SE10 9NF | London | Romney Rd | NULL | 44 20 8858 4422 | https://www.rmg.co.uk/national-maritime-museum | 51.481111 -0.005556


In [27]:
# INTERFACE!

In [28]:
!pip install tkintermapview
!pip install Pillow

In [29]:
from tkinter import *
from SPARQLWrapper import SPARQLWrapper, JSON
import webbrowser
import tkintermapview
from PIL import ImageTk, Image
import requests
from io import BytesIO

In [30]:
def fetch_data_from_query(search_query, att_names):
    
    qres = inferred_model.query(
        search_query,
        initNs={
            'rdf': rdf,
            'foaf': foaf,
            'dbr': dbr,
            'dbo': dbo,
            'dbp': dbp,
            'rdfs': rdfs,
            'georss': 'http://www.georss.org/georss/'
        },
    )


    data = []
    for row in qres:
        obj = {}
        for idx, att_name in enumerate(att_names):
            value = row[idx]
            obj[att_name] = None if value is None else str(value)
        data.append(obj)

    return data

In [39]:
def show_museum_details(museum_name):
    museum_window = Toplevel()  # Create a new top-level window
    museum_window.title(f"{museum_name}")

    # Create a Canvas widget for scrolling
    canvas = Canvas(museum_window)
    canvas.pack(side=LEFT, fill=BOTH, expand=True)

    # Create a vertical scrollbar linked to the canvas
    scrollbar = Scrollbar(museum_window, orient="vertical", command=canvas.yview)
    scrollbar.pack(side=RIGHT, fill="y")

    # Configure the canvas to use the scrollbar
    canvas.configure(yscrollcommand=scrollbar.set)

    # Create a frame inside the canvas to hold all the content
    content_frame = Frame(canvas)
    canvas.create_window((0, 0), window=content_frame, anchor="nw")

    query = f"""
            PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>

            SELECT ?abstract ?country ?state ?city ?address ?postal ?phone ?url ?coords ?thumbnail ?museum
            WHERE {{
                OPTIONAL {{
                    SERVICE <https://dbpedia.org/sparql> {{
                        ?museumDbo a dbo:Building ;
                            rdfs:label "{museum_name}"@en ;
                            georss:point ?coords ;
                            dbo:thumbnail ?thumbnail ;
                            dbo:abstract ?abstract .
                    }}
                }}
                ?museum
                    rdf:type dbo:Museum ;
                    foaf:name "{museum_name}" ;
                .
                OPTIONAL {{?museum dbp:country/foaf:name ?country .}}
                OPTIONAL {{?museum :state ?state .}}
                OPTIONAL {{?museum :city ?city .}}
                OPTIONAL {{?museum :address ?address .}}
                OPTIONAL {{?museum :postal ?postal .}}
                OPTIONAL {{?museum :phone ?phone .}}
                OPTIONAL {{?museum foaf:homepage ?url .}}
            }}
            LIMIT 1
        """
    
    att_names = ['abstract', 'country', 'state', 'city', 'address', 'postal', 'phone', 'url', 'coord', 'thumbnail', 'uri']
    museum_info = fetch_data_from_query(query, att_names)
    museum_info = museum_info[0]
    
    # Title
    title_label = Label(content_frame, text=museum_name, font=("Arial", 12))
    title_label.pack(pady=10)

    # Image
    if museum_info['thumbnail']:
        response = requests.get(museum_info['thumbnail'])
        if response.status_code == 200:
            thumbnail = Image.open(BytesIO(response.content))
            test = ImageTk.PhotoImage(thumbnail)
            label1 = Label(content_frame, image=test)
            label1.image = test
            label1.pack(pady=10)
    
    #Abstract
    #if museum_info['abstract']:
        #contact_label = Label(content_frame, text=museum_info['abstract'], font=("Arial", 10))
        #contact_label.pack(pady=10)
    
    # Contact
    contact_title_label = Label(content_frame, text=f"Contact:", font=("Arial", 12))
    contact_title_label.pack(pady=10)
    address = (museum_info['address'] if museum_info['address'] else "---") + ", " + (museum_info['city'] if museum_info['city'] else "---") + ", " + (museum_info['state'] if museum_info['state'] else "---") + ", " + (museum_info['country'] if museum_info['country'] else "---")
    contact_info = f"Phone: {museum_info['phone']}\nWebsite: {museum_info['url']}\nAddress: {address}"
    contact_label = Label(content_frame, text=contact_info, font=("Arial", 10))
    contact_label.pack(pady=10)

    # Display museum details (other fields from `museum_info` can also be added here)
    more_info_label = Label(content_frame, text="URI:", font=("Arial", 10))
    more_info_label.pack(pady=5)
    
    if museum_info['coord'] or (museum_info['address'] and museum_info['city'] and museum_info['state'] and museum_info['country']):
        map_label = LabelFrame(content_frame)
        map_label.pack(pady=20)
        
        map_widget = tkintermapview.TkinterMapView(map_label, width=800, height=600, corner_radius=0)
        
        if museum_info['coord']:
            lat, lon = map(float, museum_info['coord'].split())
            map_widget.set_position(lat, lon)
        else:
            map_widget.set_address(address)
        
        map_widget.pack()
    else:
        label = Label(content_frame, text="No info available to create a map!", font=("Arial", 12))
        label.pack(pady=10)

    painting_list = get_list_of_paintings(museum_name)
    print(painting_list)
    
    # Populate the scrollable frame with data
    for style in painting_list.keys():
        style_label = Label(content_frame, text=style, font=("Arial", 14, "bold"), pady=5)
        style_label.pack(anchor="w")
        
        for painting in painting_list[style]:
            link = Label(content_frame, text=painting['title'], fg="blue", cursor="hand2", pady=2)
            link.pack(anchor="w")
            link.bind("<Button-1>", lambda e, uri=painting['uri']:show_painting_details(uri))
    
    # Update the scrollable region of the canvas
    content_frame.update_idletasks()
    canvas.config(scrollregion=canvas.bbox("all"))

In [56]:
def create_gui(museums_by_country):
    root = Tk()
    root.title("Museums by Country")

    canvas = Canvas(root)
    scrollbar = Scrollbar(root, orient="vertical", command=canvas.yview)
    scrollable_frame = Frame(canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
    )

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)

    canvas.pack(side="left", fill="both", expand=True)
    scrollbar.pack(side="right", fill="y")

    # Populate the scrollable frame with data
    for country in museums_by_country.keys():
        country_label = Label(scrollable_frame, text=country, font=("Arial", 14, "bold"), pady=5)
        country_label.pack(anchor="w")
        
        for museum in museums_by_country[country]:
            link = Label(scrollable_frame, text=museum, fg="blue", cursor="hand2", pady=2)
            link.pack(anchor="w")
            link.bind("<Button-1>", lambda e, name=museum: show_museum_details(name))

    root.mainloop()

In [57]:
def get_list_of_paintings(museum_name):
    q = f"""
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>
        SELECT ?workLocal ?title ?style
        WHERE {{
            SERVICE <https://dbpedia.org/sparql> {{
                ?workDbo a dbo:Work ;
                         dbp:title ?title ;
                         dbo:museum/rdfs:label "{museum_name}"@en .
            }}
            OPTIONAL {{
                ?workLocal
                    rdf:type dbo:Painting ;
                    foaf:name ?titleLocal ;
                    dbp:style/foaf:name ?style ;
                .
                FILTER (STR(?title) = STR(?titleLocal))
            }}
        }}
    """

    painting_list = fetch_data_from_query(q, ['painting_uri', 'title','style'])
    
    organized = {}
    for painting in painting_list:
        uri = painting['painting_uri']
        title = painting['title'] if painting['style'] else 'Untitled'
        style = painting['style'] if painting['style'] else 'Unclassified'
        
        if style not in organized:
            organized[style] = []
        obj = {'title': title, 'uri': uri}
        print(uri)
        organized[style].append(obj)
    
    return organized

In [58]:
def organize_by_country(museums):
    organized = {}
    for museum in museums:
        country = museum['country']
        museum_name = museum['museum_name']
        
        if country not in organized:
            organized[country] = []
        organized[country].append(museum_name)
    
    return organized

In [59]:
def show_painting_details(painting_uri):
    painting_window = Toplevel()  # Create a new top-level window
    painting_window.title(f"{painting_uri}")

    # Create a Canvas widget for scrolling
    canvas = Canvas(painting_window)
    canvas.pack(side=LEFT, fill=BOTH, expand=True)

    # Create a vertical scrollbar linked to the canvas
    scrollbar = Scrollbar(painting_window, orient="vertical", command=canvas.yview)
    scrollbar.pack(side=RIGHT, fill="y")

    # Configure the canvas to use the scrollbar
    canvas.configure(yscrollcommand=scrollbar.set)

    # Create a frame inside the canvas to hold all the content
    content_frame = Frame(canvas)
    canvas.create_window((0, 0), window=content_frame, anchor="nw")

    query = f"""
            PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>

            SELECT ?title ?artist ?style ?museum ?thumbnail
            WHERE {{
                SERVICE <https://dbpedia.org/sparql> {{
                    ?workDbo a dbo:Work ;
                        dbo:museum/rdfs:label ?museum ;
                        dbo:thumbnail ?thumbnail .
                }}
                <{painting_uri}> foaf:name ?title ;
                    dbp:artist/foaf:name ?artist ;
                    dbp:style/foaf:name ?style .
            }}
            LIMIT 1
        """
    
    att_names = ['title', 'artist', 'style', 'museum', 'thumbnail']
    painting_info = fetch_data_from_query(query, att_names)
    painting_info = painting_info[0]
    print(painting_info)
    
     # Title
    title_label = Label(content_frame, text=painting_info['title'], font=("Arial", 12))
    title_label.pack(pady=10)

    # Image
    if painting_info['thumbnail']:
        response = requests.get(painting_info['thumbnail'])
        if response.status_code == 200:
            thumbnail = Image.open(BytesIO(response.content))
            test = ImageTk.PhotoImage(thumbnail)
            label1 = Label(content_frame, image=test)
            label1.image = test
            label1.pack(pady=10)
    
    # Info
    contact_label = Label(content_frame, text=f"Artist: {painting_info['artist']}", font=("Arial", 10))
    contact_label.pack(pady=5)
    contact_label = Label(content_frame, text=f"Style: {painting_info['style']}", font=("Arial", 10))
    contact_label.pack(pady=5)
    contact_label = Label(content_frame, text=f"Museum: {painting_info['museum']}", font=("Arial", 10))
    contact_label.pack(pady=5)

    # URI
    more_info_label = Label(content_frame, text=f"URI: {painting_uri}", font=("Arial", 10))
    more_info_label.pack(pady=5)
    
        
        
    #link = Label(content_frame, text=painting, fg="blue", cursor="hand2", pady=2)
    #link.pack(anchor="w")
    #link.bind("<Button-1>", lambda e, name=museum:show_museum_details(name))
    
    # Update the scrollable region of the canvas
    content_frame.update_idletasks()
    canvas.config(scrollregion=canvas.bbox("all"))

In [ ]:
q = """
        PREFIX : <http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#>

        SELECT ?country ?MuseumName
        WHERE {
            ?museum rdf:type dbo:Museum ;
                    foaf:name ?MuseumName .
            OPTIONAL { ?museum dbp:country/foaf:name ?country . }
        }
        ORDER BY ?country
    """
att_names = ['country', 'museum_name']
data = fetch_data_from_query(q, att_names)
museums_by_country = organize_by_country(data)
create_gui(museums_by_country)

http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#198427_Annunciation
http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#29762_Annunciation
http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#168000_Annunciation
None
None
None
http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#191514_PortraitofaYoungMan
http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#7009_PortraitofaYoungMan
http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#27621_PortraitofaYoungMan
http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#21399_PortraitofaYoungMan
http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#7865_PortraitofaYoungMan
http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#21198_PortraitofaYoungMan
http://www.semanticweb.org/ericarfs/ontologies/2024/10/famouspaintings#21369_PortraitofaYoungMan
http://www.semanticweb.org/ericarfs